# 控制器：Controllers
- 继承于odoo.http.Controllers

# API
## 路由
### odoo.http.route(route=None, **kw)
- 参数说明：
    - route：路由
    - type：请求类型：json和http
    - auth：认证：user（当前用户的权限），public（公共用户的权限），None（没有任何权限）
    - methods：请求方法列表：['GET', 'POST', 'HEAD']
    - cors:跨域
    - csrf：默认为True
        - 大多数是POST请求会用到这个csrf的csrf_token，如果post请求中表单form是python中生成的，那么使用request.csrf_token()就可以获取到。如果是在js中生成的（Qweb）那么就在web.core中。require('web.core').csrf_token

## Request
### odoo.http.WebRequest(httprequest)：
- 所有Odoo Web请求类型的父类，主要处理请求对象的 初始化和设置(分派本身必须由子类处理)
- httprequest:一个包装好的werkzeug请求对象
- 属性
    - params：请求参数的映射，通常并不有用，因为它们作为关键字参数直接提供给处理程序方法
    - cr：游标，如果auth为None，使用它会报错
    - context：当前请求的上下文值映射
    - env：当前请求的环境
    - session：OpenERPSession保存当前HTTP会话的HTTP会话数据
    - db：当前请求的数据库，如果auth为None，则为空
    - csrf_token(time_limit=None)：
    
### odoo.http.HttpRequest(*args)
- 专门来处理http的请求类行
- 匹配的路由参数、查询字符串参数、表单参数和文件作为关键字参数传递给处理程序方法。
- 返回值类型：假值204（没有内容),werkzeug Response对象，str或者Unicode（会被包装在Response中）并被解释为HTMl
- 方法：
    - make_response(data, headers=None, cookies=None)
        - 非HTML响应，或带有自定义响应头或cookie的HTML响应。
        - 如果返回的是非HTML数据，处理程序可以只返回想要以字符串形式发送的页面的HTML标记，但它们需要创建一个完整的Response对象，否则返回的数据将不能被客户机正确解释。
        - 参数解释：
            - data：响应body
            - headers：给response设置请求头，形式：[(name, value)]
            - cookies:给客户端设置的cookies
    - not_found(description=None)
        - 404的快捷返回方法
    - render(template, qcontext=None, lazy=True, **kw)
        - QWeb模板的延迟渲染。
        - 实际的模板渲染发生在调用结束时进行。所以模板和qcontext可以被静态response修改甚至替换。
        - 参数解释：
            - lazy：模板呈现是否应该延迟到最后一刻
            - **kw:转发到werkzeug Response对象的关键字
### odoo.http.JsonRequest(*args)
- method会被忽略
- 请求的参数必须是json类型
- 返回的类型是jsonRpc类型，并被包装在jsonRpc Response中
    - jsonRpc Response对象
        - jsonRpc是一种无状态、轻量级远程过程调用(RPC)协议
        - 返回参数：
            - jsonrpc：表示版本
            - result：如果请求成功就返回，请求错误则绝对不反悔
            - error：与上相反
                - error对象的定义
                    - code，message，data
            - id：和请求中的id相同
    - 示例：
            请求类型
            {"jsonrpc": "2.0",
             "method": "call",
             "params": {"context": {},
                        "arg1": "val1" },
             "id": null}
             
             返回类型
             {"jsonrpc": "2.0",
             "result": { "res1": "val1" },
             "id": null}
             
             返回错误类型
             {"jsonrpc": "2.0",
             "error": {"code": 1,
                   "message": "End user error message.",
                   "data": {"code": "codestring",
                        "debug": "traceback" } },
             "id": null}
             
 ## Response
 ### odoo.http.Response(*args, **kw)
 - 除了werkzeug.wrapper.response参数之外，这个类的构造函数还可以为QWeb Lazy Rendering接受以下附加参数
     - template
     - qcontext
     - uid:用于ir.ui.view渲染调用的用户id, 为None则使用请求的用户(默认)
 - 方法：
     - render():呈现响应的模板，返回结果
     - flatten():强制响应模板的呈现，将结果设置为响应体并取消模板